<a href="https://colab.research.google.com/github/OConnorw/BIA_DR/blob/main/Ennis_DR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')


train_dir = '/content/drive/MyDrive/diabetic-retinopathy/diabetic-retinopathy/0/'
test_dir = '/content/drive/MyDrive/diabetic-retinopathy/diabetic-retinopathy/'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 0 images belonging to 4 classes.
Found 0 images belonging to 4 classes.
Found 96 images belonging to 5 classes.


3/3 [==============================] - 15s 4s/step - loss: 1.3171 - accuracy: 0.4271
Test accuracy: 0.4270833432674408


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(299, 299),
        batch_size=32,
        class_mode='categorical',
        classes='auto',
        subset='training')

valid_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(299, 299),
        batch_size=32,
        class_mode='categorical',
        classes='auto',
        subset='validation')

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')

In [ ]:

# Create model using pre-trained InceptionV3 base
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import InceptionV3
base_model = InceptionV3(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
for layer in base_model.layers:
    layer.trainable = False
x = layers.Flatten()(base_model.output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(5, activation='softmax')(x)
model = keras.Model(inputs=base_model.input, outputs=x)


In [ ]:
# Compile and train model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Evaluate model on test dataset
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)
